In [1]:
from keras.models import *
from keras.layers import *
from keras.optimizers import Nadam
import h5py
import numpy as np
from generator import DriveDataGenerator

DATA_FILE = 'data.h5df'

C:\Users\CBlagden\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with h5py.File(DATA_FILE, 'r') as f:
    
    images_training = list(f['images_training'])
    labels_training = np.array(f['label_training'])
    
    images_testing = list(f['images_testing'])
    labels_testing = np.array(f['label_testing'])

In [4]:
activation = 'relu'

# How many images in past at time to process
num_images_input = 4

# Multiply num_images_input by 3 since they're BGR
pic_input = Input(shape=(128, 128, num_images_input*3))

img_stack = Conv2D(32, (3, 3), name='convolution0', padding='same', activation=activation)(pic_input)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Conv2D(32, (3, 3), name='convolution1', padding='same', activation=activation)(img_stack)
img_stack = MaxPooling2D(pool_size=(2,2))(img_stack)
img_stack = Conv2D(16, (3, 3), name='convolution2', padding='same', activation=activation)(img_stack)
img_stack = Flatten()(img_stack)
img_stack = Dropout(0.2)(img_stack)

dense_stack = Dense(64, activation=activation, name='dense0')(img_stack)
dense_stack = Dropout(0.2)(dense_stack)
dense_stack = Dense(10, activation=activation, name='dense2')(dense_stack)
dense_stack = Dropout(0.2)(dense_stack)
dense_stack = Dense(1, name='output', activation='sigmoid')(dense_stack)

adam = Nadam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model = Model(inputs=[pic_input], outputs=dense_stack)
model.compile(optimizer=adam, loss='mse')

In [ ]:
model.summary()

In [ ]:
training_generator = DriveDataGenerator(images_training, labels_training)
validation_generator = DriveDataGenerator(images_testing, labels_testing)

model.fit_generator(generator=training_generator, validation_data=validation_generator,
                    verbose=1, epochs=300)

Epoch 1/300


In [6]:
save_model(model, 'model.h5df')